## Explore PubAG API

In [1]:
import requests
import json

In [2]:
API_KEY = "jGgEv0rYItqngfgOd9eF9HN5GoCDRxTzB64slFdL"

In [28]:
with open('pubag_abs_dict.json', 'r') as read_file:
    abstract_dict = json.load(read_file)

keywords = "precision+agriculture"
response = requests.get("https://api.nal.usda.gov/pubag/rest/search/?query={}&per_page=100&api_key={}".format(keywords, API_KEY)).json()
totalpages = response["request"]["totalPages"]

for i in range (totalpages):
    response = requests.get("https://api.nal.usda.gov/pubag/rest/search/?query={}&page={}&per_page=100&api_key={}".format(keywords, i+1, API_KEY)).json()
    for j in range (len(response['resultList'])):
        abs_id = response['resultList'][j]['id']
        if abs_id not in abstract_dict['id']:
            try:
                abstract_dict['id'].append(abs_id)
                abstract_dict['titles'].append(response['resultList'][j]['title'])
                abstract_dict['abstracts'].append(response['resultList'][j]['abstract'])
                abstract_dict['authors'].append((response['resultList'][j]['author']))
                abstract_dict['keywords'].append((keywords))
                abstract_dict['subject'].append((response['resultList'][j]['subject']))
                abstract_dict['date'].append((response['resultList'][j]['date']))
                abstract_dict['sources'].append((response['resultList'][j]['journal']))
            except:
                print(abs_id, " didn't work")
                                             
with open('pubag_abs_dict.json', 'w+') as f:
    json.dump(abstract_dict, f) 

18552  didn't work
2846665  didn't work
1552676  didn't work
2187318  didn't work
2187319  didn't work
2202449  didn't work
2837784  didn't work
2641343  didn't work
17630  didn't work
3565536  didn't work
2127885  didn't work
2865167  didn't work
2123647  didn't work
2271001  didn't work
2207168  didn't work
2271828  didn't work
2246731  didn't work
2774897  didn't work
2846847  didn't work
2865075  didn't work
3067709  didn't work
2207166  didn't work
2235481  didn't work
35763  didn't work
1998729  didn't work
1537734  didn't work
3103144  didn't work
2864932  didn't work
3110044  didn't work
2020776  didn't work
20785  didn't work
4228982  didn't work
2876319  didn't work
2016231  didn't work
2062307  didn't work
1537603  didn't work
1552670  didn't work
1381018  didn't work
1381019  didn't work
2186459  didn't work
2071325  didn't work
2805534  didn't work
1537446  didn't work
1537584  didn't work
3055992  didn't work
1380458  didn't work
2775006  didn't work
2864981  didn't work


In [29]:
print(len(abstract_dict['id']))

2938


In [30]:
with open('pubag_abs_dict.json', 'r') as read_file:
    abstract_dict = json.load(read_file)
print(len(abstract_dict['id']))

2938
